# Functionality:
extract rest inf with the same date of skb task

## Import Packages 

In [1]:
import os, sys
import numpy as np
import glob
import scipy.io as sio
import pandas as pd

from numpy.fft import fft, fftfreq
from scipy.signal import spectrogram
from datetime import datetime

import matplotlib.pyplot as plt

In [2]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# import functions in util folder
from util.folder_extract import exp_subfolders, code_corresfolder
from util.parsePDCondition import parsePDCondition_Pinky

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/parsePDCondition.ipynb


## exp subfolders

In [3]:
datafolder, _, pipelinefolder, _= exp_subfolders()

## code_corresfolder

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameter &save setup

In [6]:
animal = 'Pinky'

In [7]:
savefolder = corresfolder
savefilename = animal + '_restinf.csv'
savefile = os.path.join(savefolder, savefilename)

## read mastersheet & skbinf 

In [8]:
# read mastersheet file
file_mastersheet = os.path.join(datafolder, 'pinky_mastersheet.xlsx')
df_master = pd.read_excel(file_mastersheet, sheet_name='MasterList')


# extract outputfolder, task, and tdtblock
master_outputfolder = df_master['OutputFolderName']
master_task = df_master['Brief Description']
master_tdtblock = df_master['TDT Block']

In [9]:
# read skbinf file
file_skbinf = os.path.join(datafolder, 'pinky_skbinf.csv')
df_skbinf = pd.read_csv(file_skbinf,
                        dtype={'dateofexp': str, 'bktdt': int})


# skbinf_dateofexp(str), skbinf_bktdt (int)
skbinf_dateofexp, skbinf_bktdt = df_skbinf['dateofexp'], df_skbinf['bktdt']

# Marked Yes under YingUsed
skbinf_yingused, skbinf_extremelyhighValue = df_skbinf['YingUsed'], df_skbinf['ExtramelyHigh']

## extract rest inf with the same date of skb task

In [10]:
## deal with each date of exp for yingusd without high value
rest_dateofexp, rest_bktdt, rest_pdCondition = [], [], []
for date in skbinf_dateofexp[(skbinf_yingused == 'Yes') & (skbinf_extremelyhighValue != 'Yes')]:
    
    # dateofexp(datetime object)
    dateofexp = datetime.strptime(date, '%y%m%d')
    
    # date of exp subfolder
    subfolder_dateofexp = animal + '_' + dateofexp.strftime('%m%d%y')
    
    # extract the tdt block number of resting in dateofexp
    mask = (master_outputfolder == subfolder_dateofexp) & (master_task == 'Resting')
    bktdt = int(master_tdtblock[mask])
    
    
    # append to rest_dateofexp, rest_bktdt
    rest_dateofexp.append(dateofexp.strftime('%y%m%d'))
    rest_bktdt.append(bktdt)
    rest_pdCondition.append(parsePDCondition_Pinky(dateofexp))
    
    del dateofexp, subfolder_dateofexp, mask, bktdt

## save

In [11]:
df = pd.DataFrame({'dateofexp': rest_dateofexp,
                   'bktdt': rest_bktdt,
                   'pdCondition': rest_pdCondition})
df.to_csv(savefile)
print('save to ' + savefile)

save to /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/0_dataPrep/0_restinf/Pinky_restinf.csv
